In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']

In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_db_supra_fresh.f_spr_daily_sale 
-- CLUSTER BY store_id, product_id 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH
table_date AS (
    SELECT EXPLODE(SEQUENCE(DATE_SUB(CURRENT_DATE(),  40 ), DATE_SUB(CURRENT_DATE(),  1 ))) date_gen
),
-- Bộ sản phẩm combo OLALA
product_olala_bom AS (
  SELECT DISTINCT
    a.product_id,
    a.component_id,
    a.base_quantity,
    a.component_quantity,
    b.manufacturer_id
  FROM
    {catalog_name}.udp_wcm_gold_vcm_dwh.d_product_bom a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.component_id = b.product_id
  WHERE
    b.brand_id = '15871' AND
    b.manufacturer_id = '1915' AND
    UPPER(mch4_desc) LIKE '%XÚC XÍCH%'
),
-- Danh sách Product MDL
product_mdl AS (
    SELECT *
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product
    WHERE
        product_id IN(SELECT DISTINCT product_id FROM {catalog_name}.udp_wcm_gold_vcm_db_supra_fresh.d_winare_mdl_special_sku_add)
        OR manufacturer_id IN ('1915', '2325', '2640') 
        OR product_id IN (SELECT DISTINCT product_id FROM product_olala_bom)
        OR (manufacturer_id ='1012' AND brand_name IN ('HEO CAO BỒI','PONNIE'))
),
asm_active AS (
    SELECT
        a.store_id,
        a.product_id,
        p.manufacturer_id,
        IFNULL(IF(s.store_open_date > CURRENT_DATE - 14, 'YES', 'NO'),'NO') is_new_store,
        s.city,
        s.size_v,
        MIN(a.start_date) asm_active_min_start_date
    FROM
        {catalog_name}.udp_wcm_gold_vcm_dwh.d_assortment_scd a
        INNER JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store s ON a.store_id = s.store_id
        INNER JOIN product_mdl p ON a.product_id = p.product_id
    WHERE
        a.status = 'ACTIVE'
        AND s.business_unit in ('1500', '2000')
        AND s.store_status_desc in ("Đang hoat động","Đang hoạt động","Sắp đóng cửa")
        AND UPPER(s.store_name) NOT LIKE '[BLOCK]%'
        AND UPPER(s.store_name) NOT LIKE 'OUTLET%'
        AND UPPER(s.store_name) NOT LIKE '%HUB%'
        AND UPPER(s.store_name) NOT LIKE '%ONLINE%'
    GROUP BY 1,2,3,4,5,6
),
base_sale_qty AS (
    SELECT
        a.calday,
        a.store_id,
        IF(c.product_id IS NOT NULL, c.component_id, a.product_id) product_id,
        IF(c.product_id IS NOT NULL, c.manufacturer_id, b.manufacturer_id) manufacturer_id,
        SUM(a.base_sale_qty) base_sale_qty
    FROM {catalog_name}.udp_wcm_gold_vcm_dwh.f_sale_prod a
        LEFT JOIN product_mdl b ON a.product_id = b.product_id
        LEFT JOIN product_olala_bom c ON a.product_id = c.product_id
    WHERE calday >= DATE_SUB(CURRENT_DATE(),  40 ) AND calday < CURRENT_DATE()
        AND a.product_id IS NOT NULL
        AND b.manufacturer_id IS NOT NULL
        AND b.product_id IS NOT NULL
        AND (a.promotion NOT IN ('1000018319','1000062785') or a.promotion IS NULL)
    GROUP BY 1,2,3,4
),
store_sku_full AS (
    SELECT
        date_gen calday,
        a.store_id,
        a.product_id,
        a.manufacturer_id
    FROM (
        SELECT DISTINCT
            store_id,
            product_id,
            manufacturer_id
        FROM asm_active
    ) a, table_date b
)
, temp_0 AS (
    SELECT
        a.calday,
        a.store_id,
        a.product_id,
        IFNULL(b.manufacturer_id,a.manufacturer_id) AS manufacturer_id,
        c.is_new_store,
        c.city,
        c.size_v,
        IF(b.base_sale_qty IS NULL OR b.base_sale_qty <0, 0, b.base_sale_qty) base_sale_qty,
        IFNULL(c.asm_active_min_start_date, DATE_SUB(CURRENT_DATE(),  40 )) asm_active_min_start_date,
        d.dayofweek
    FROM store_sku_full a
        LEFT JOIN base_sale_qty b ON a.calday = b.calday AND a.store_id = b.store_id AND a.product_id = b.product_id
        INNER JOIN asm_active c ON a.store_id = c.store_id AND a.product_id = c.product_id
        LEFT JOIN {catalog_name}.udp_wcm_silver_dim.d_time d ON a.calday = d.calday
),
-- Adhoc tránh sale ngày bão
adhoc_00 AS (
  SELECT a.*
  FROM
    temp_0 a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_store b ON a.store_id = b.store_id
  WHERE NOT (
    b.mien IN ('Miền Bắc', 'Miền Trung') AND
    a.calday BETWEEN '2024-09-04' AND '2024-09-15'
  )
),
adhoc_01 AS (
  SELECT a.*
  FROM adhoc_00 a
  QUALIFY DENSE_RANK() OVER (PARTITION BY store_id ORDER BY a.calday DESC) <= 28
)
, temp_1 AS (
    SELECT *, IF(calday >= asm_active_min_start_date, 1, 0) is_sum
    FROM adhoc_01
)
, temp_2 AS (
    SELECT
        a.store_id,
        a.product_id,
        a.manufacturer_id,
        b.base_uom base_unit,
        a.asm_active_min_start_date,
        a.is_new_store,
        a.city,
        a.size_v,
        TRY_DIVIDE(SUM(base_sale_qty), COUNT(a.calday)) avg_sold_qty_28d,
        TRY_DIVIDE(SUM(IF(is_new_store = "NO",base_sale_qty,0)), COUNT(IF(is_new_store = "NO", a.calday, NULL))) avg_sold_qty_28d_oldstore,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 2 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 2 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4mon,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 3 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 3 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4tue,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 4 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 4 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4wed,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 5 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 5 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4thu,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 6 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 6 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4fri,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 7 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 7 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4sat,
        TRY_DIVIDE(SUM(IF(a.dayofweek = 1 AND is_new_store = "NO", base_sale_qty, 0)), COUNT(IF(a.dayofweek = 1 AND is_new_store = "NO", a.calday, NULL))) avg_sold_qty_4sun
    FROM temp_1 a
        LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b ON a.product_id = b.product_id
    WHERE is_sum = 1
    GROUP BY 1,2,3,4,5,6,7,8
)
, avg_sold AS (
    SELECT
        store_id,
        product_id,
        manufacturer_id,
        base_unit,
        asm_active_min_start_date,
        avg_sold_qty_28d,
        avg_sold_qty_28d_oldstore,
        is_new_store,
        city,
        size_v,
        IFNULL(avg_sold_qty_4mon, avg_sold_qty_28d) avg_sold_qty_4mon,
        IFNULL(avg_sold_qty_4tue, avg_sold_qty_28d) avg_sold_qty_4tue,
        IFNULL(avg_sold_qty_4wed, avg_sold_qty_28d) avg_sold_qty_4wed,
        IFNULL(avg_sold_qty_4thu, avg_sold_qty_28d) avg_sold_qty_4thu,
        IFNULL(avg_sold_qty_4fri, avg_sold_qty_28d) avg_sold_qty_4fri,
        IFNULL(avg_sold_qty_4sat, avg_sold_qty_28d) avg_sold_qty_4sat,
        IFNULL(avg_sold_qty_4sun, avg_sold_qty_28d) avg_sold_qty_4sun
    FROM temp_2
)
, avg_sold2 AS (
	SELECT
		a.*,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4mon,0)) OVER ( PARTITION BY store_id, manufacturer_id),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_mon,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4tue,0)) OVER ( PARTITION BY store_id, manufacturer_id),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_tue,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4wed,0)) OVER ( PARTITION BY store_id, manufacturer_id ),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_wed,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4thu,0)) OVER ( PARTITION BY store_id, manufacturer_id ),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_thu,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4fri,0)) OVER ( PARTITION BY store_id, manufacturer_id ),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_fri,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4sat,0)) OVER ( PARTITION BY store_id, manufacturer_id ),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_sat,
		TRY_DIVIDE(SUM(IF(is_new_store = "NO",avg_sold_qty_4sun,0)) OVER ( PARTITION BY store_id, manufacturer_id ),SUM(avg_sold_qty_28d_oldstore) OVER ( PARTITION BY store_id, manufacturer_id )) fc_rate_sun
  FROM avg_sold a
)
, avg_sold3 AS (
  SELECT 	*,
      avg_sold_qty_28d * fc_rate_mon fc_sold_qty_mon,
      avg_sold_qty_28d * fc_rate_tue fc_sold_qty_tue,
      avg_sold_qty_28d * fc_rate_wed fc_sold_qty_wed,
      avg_sold_qty_28d * fc_rate_thu fc_sold_qty_thu,
      avg_sold_qty_28d * fc_rate_fri fc_sold_qty_fri,
      avg_sold_qty_28d * fc_rate_sat fc_sold_qty_sat,
      avg_sold_qty_28d * fc_rate_sun fc_sold_qty_sun
  FROM avg_sold2)
, sold_city AS (
    SELECT
        a.city, a.size_v,product_id,
        AVG(avg_sold_qty_28d)  fc_sold_qty_28d_city,
        AVG(a.fc_sold_qty_mon) fc_sold_qty_mon_city,
        AVG(a.fc_sold_qty_tue) fc_sold_qty_tue_city,
        AVG(a.fc_sold_qty_wed) fc_sold_qty_wed_city,
        AVG(a.fc_sold_qty_thu) fc_sold_qty_thu_city,
        AVG(a.fc_sold_qty_fri) fc_sold_qty_fri_city,
        AVG(a.fc_sold_qty_sat) fc_sold_qty_sat_city,
        AVG(a.fc_sold_qty_sun) fc_sold_qty_sun_city
    FROM avg_sold3 a
    WHERE is_new_store = 'NO'
    GROUP BY 1,2,3
)
, avg_sold4 AS (
  SELECT
    a.* EXCEPT(fc_sold_qty_mon,fc_sold_qty_tue,fc_sold_qty_wed,fc_sold_qty_thu,fc_sold_qty_fri,fc_sold_qty_sat, fc_sold_qty_sun),
      fc_sold_qty_mon fc_sold_qty_mon_0,
      fc_sold_qty_tue fc_sold_qty_tue_0,
      fc_sold_qty_wed fc_sold_qty_wed_0,
      fc_sold_qty_thu fc_sold_qty_thu_0,
      fc_sold_qty_fri fc_sold_qty_fri_0,
      fc_sold_qty_sat fc_sold_qty_sat_0,
      fc_sold_qty_sun fc_sold_qty_sun_0,
      IF(is_new_store = 'NO',avg_sold_qty_28d, IF(IFNULL(fc_sold_qty_28d_city,0)=0,1,fc_sold_qty_28d_city)) fc_sold_qty_28d,
      IF(is_new_store = 'NO',fc_sold_qty_mon, IF(IFNULL(fc_sold_qty_mon_city,0)=0,1,fc_sold_qty_mon_city))  fc_sold_qty_mon,
      IF(is_new_store = 'NO',fc_sold_qty_tue, IF(IFNULL(fc_sold_qty_tue_city,0)=0,1,fc_sold_qty_tue_city))  fc_sold_qty_tue,
      IF(is_new_store = 'NO',fc_sold_qty_wed, IF(IFNULL(fc_sold_qty_wed_city,0)=0,1,fc_sold_qty_wed_city))  fc_sold_qty_wed,
      IF(is_new_store = 'NO',fc_sold_qty_thu, IF(IFNULL(fc_sold_qty_thu_city,0)=0,1,fc_sold_qty_thu_city))  fc_sold_qty_thu,
      IF(is_new_store = 'NO',fc_sold_qty_fri, IF(IFNULL(fc_sold_qty_fri_city,0)=0,1,fc_sold_qty_fri_city))  fc_sold_qty_fri,
      IF(is_new_store = 'NO',fc_sold_qty_sat, IF(IFNULL(fc_sold_qty_sat_city,0)=0,1,fc_sold_qty_sat_city))  fc_sold_qty_sat,
      IF(is_new_store = 'NO',fc_sold_qty_sun, IF(IFNULL(fc_sold_qty_sun_city,0)=0,1,fc_sold_qty_sun_city))  fc_sold_qty_sun
  FROM avg_sold3 a
  LEFT JOIN sold_city c
  ON a.city = c.city
  AND a.product_id = c.product_id
  AND a.size_v = c.size_v
)
SELECT * FROM avg_sold4 a
;
""")